# Sentiment analysis para clasificar críticas de películas

## Introducción
La idea de este notebook es establecer un método general para aplicar _sentiment analysis_ adaptable a otros datasets.

Primero vamos a ver las diferentes features que podemos extraer de un texto relativas a _sentiment analysis_. Construiremos un extractor de features generalizado que pueda valer para la mayoría de problemas

Luego aplicaremos las features a diferentes modelos, como _Support Vector Machines_ (__SVM__) o un clasificador _Naive Bayes_.

Lo interesante de esta estrategia es que nos permitirá saber automáticamente cuáles son las mejores features para nuestro problema mediante una búsqueda de parámetros.


## Preparación

Las bibliotecas de python usadas para este proyecto son `nltk` y `scikit-learn`. Antes de empezar hay que descargar los corpus necesarios para las diferentes utilidades de `nltk`

In [337]:
import nltk
nltk.download('tagsets')
# TODO Faltan aqui un par de downloads

[nltk_data] Downloading package tagsets to /home/marhs/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

## Sentiment analysis

_Sentiment analysis_ es la extracción/análisis de opiniones subjetivas (no objetivas) sobre un texto. Puede ser la polarización del texto (si habla bien o mal de algún tema), clasificación de sentimientos: si el emisor del mensaje está contento, enfadado, triste, etc, diferentes actitudes hacia un tema, como agresividad o calma.

El uso de este tipo de técnicas en la vida real es muy amplio, ya que permite automatizar procesos de naturaleza ambigua como:

* Medidor de opiniones sobre una tema en particular en redes sociales (twitter, facebook)
* Identificación de contenido inapropiado (insultos, _flames_) para automoderación de comunidades
* Detección de sesgos en medios de comunicación como periódicos

En este caso vamos a clasificar críticas de películas hechas por usuarios (en inglés). La idea es que si tenemos críticas escritas en foros, twitter, etc podremos clasificar si una película es buena o mala en base al la reacción del público en general.

### Sentiment analysis como un problema de clasificación

Para simplificar el problema y poder aplicar técnicas ya conocidas de clasificación vamos a crear un modelo que clasifique las críticas de usuarios en dos categorías: buenas (1) y malas (0). Como en los problemas de clasificación en general, la idea es convertir cada documento en una serie de features y entrenar un modelo que clasifique en nuestras dos categorías (0 y 1) dadas una features.

### Vector space model

El modelo elegido para este caso es el _vector space model_ o _term vector model_. Es este modelo consideramos que un documento es una colección de palabras. Cada documento puede ser clasificado en base a la __frecuencia de palabras__ que aparecen en él. Una mala crítica dejará una frecuencia más alta de palabras malas como: _horrible, worst, bad, worthless_, mientras que una buena dejará una frecuencia alta de buenas palabras: _great, enjoyable, best_.

Hay determinadas palabras, como _"of"_ o _"the"_ que van a aparecer con una alta frecuencia en la mayoría de documentos. Para que esto no haga más difícil clasificar documentos en base a la frecuencia poodemos probar a añadir también la __frecuencia inversa__: como de frecuente es la palabra en cuestión entre todos los documentos disponibles.

Este modelo se conoce como __TF-IDF__: _Term frequency-inverse document frequency_



## Extracción de features

Aunque hemos hablado de frecuencia de palabras en los _vector space model_ podemos jugar mucho con la definición de "palabra" para generar diferentes modelos.

En procesamiento de lenguaje natural hemos visto una aproximación interesante al análisis de textos: los _n-gramas_: secuencias de n elementos del texto a analizar que nos permiten analizar por separado estructuras del texto, ya sean palabras (unigramas) o cadenas más largas de palabras (bigramas, trigramas).

Podemos generar diferentes modelos de _term frequency_ usando unigramas, bigramas, etc.

Vamos a cargar un ejemplo del conjunto de entrenamiento para ver como extraer diferentes características. Para este ejemplo vamos a cargar la review negativa \#3886 del dataset.

In [169]:
raw_text = open('data/train/neg/3886_1.txt').read()
print(raw_text)

Is this a stupid movie? You bet!! I could not find any moment in this film that was creepy or scary. Stupid moments? Plenty. Stupid characters? You bet. Bad effects? Everywhere! Rick Baker may have gone and done bigger and better things, this is not one of them. Oh well people gotta start somewhere. Dr. Ted Nelson is cheesed. He is the most whiny doctor I've ever seen. He's got a melting man running amok out in Ventura County somewhere, he's not overly happy that his wife is pregnant (probably cause she's 55 years old and weighs 90 lbs) and there's no crackers to be found anywhere. Plus he's got the not-too-helpful general on his hinder wanting to find astronaut Steve. And the local sheriff wants to know what's going on even though Mr. Nelson can't tell him anything. There also some random characters thrown in for good measure who encounter the melting man. Eventually the movie ends and out monster gets scooped into a trash can to become compost. In the end it's just what you need for 

### Unigramas y bigramas

Vamos a usar `nltk` como herramienta de __NLP__. `nltk` posee la utilidad `word_tokenize`, que dado un texto lo divide en palabras, generando así su unigrama. Con `nltk.util.ngrams` y este unigrama podemos generar n-gramas de la longitud deseada.

In [248]:
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

In [261]:
tokens = word_tokenize(raw_text)
tokens[120:140]

['lbs',
 ')',
 'and',
 'there',
 "'s",
 'no',
 'crackers',
 'to',
 'be',
 'found',
 'anywhere',
 '.',
 'Plus',
 'he',
 "'s",
 'got',
 'the',
 'not-too-helpful',
 'general',
 'on']

In [252]:
bigram = list(ngrams(tokens, n=2))
bigram[10:20]

[('I', 'could'),
 ('could', 'not'),
 ('not', 'find'),
 ('find', 'any'),
 ('any', 'moment'),
 ('moment', 'in'),
 ('in', 'this'),
 ('this', 'film'),
 ('film', 'that'),
 ('that', 'was')]

Se puede ver que `word_tokenize` ha separado todas las palabras del texto, con algunos errores. Conceptos como _not-too-helpful_ han sido imposibles de separar ya que ha interpretado los guiones como una misma palabra.

### Stemming

Es posible que nos encontremos palabras con significados muy parecidos a la hora de clasificar en _sentiment analysis_ pero que al ser palabras distintas vayan a contar por separado. Un ejemplo de esto podría ser __*happy*__ (feliz) junto a __*happier*__ (más feliz que). Ambos ejemplos aportan la misma idea: felicidad, pero si solo usamos la frecuencia de palabras van a ser analizadas por separado.

Una posible aproximación para arreglar esto es usar solo la raíces _(stems)_ de las palabras. De esta manera _happy_ y _happier_ se usarían como __*happi*__, aumentando la frecuencia del mismo concepto. También van a ayudar a transformarlo todo a minúsculas.

Los diferentes _stemmers_ disponibles en NLTK no son perfectos, pero podemos ver como funcionan sobre un conjunto de palabras que ya hemos sacado en el ejemplo anterior

In [114]:
from nltk.stem.snowball import SnowballStemmer

In [115]:
snow_stemmer = SnowballStemmer(language='english')

Vemos cada palabra con su raíz

In [265]:
stems = [(word, snow_stemmer.stem(word)) for word in tokens]
stems[20:30]

[('was', 'was'),
 ('creepy', 'creepi'),
 ('or', 'or'),
 ('scary', 'scari'),
 ('.', '.'),
 ('Stupid', 'stupid'),
 ('moments', 'moment'),
 ('?', '?'),
 ('Plenty', 'plenti'),
 ('.', '.')]

### Part of speech (POS) tagging

Por último podemos aprovechar toda la potencia que da el análisis POS. _Part of speech_ (POS) es el análisis sintáctico del texto. Nos va a permitir saber cual es el uso de cada una de las palabras(verbo, nombre, adjetivos). De esta manera podremos afinar mucho más en aquellos sitios donde sabemos que está nuestra información.

En este caso, sabemos que las opiniones se producen sobre todo en adjetivos y adverbios (bueno, malo, peor que, etc), por lo tanto podemos hacer un filtrado por este tipo de palabras.

In [124]:
from nltk import pos_tag

In [332]:
tagged_tokens = pos_tag(tokens)
tagged_tokens[10:20]

[('I', 'PRP'),
 ('could', 'MD'),
 ('not', 'RB'),
 ('find', 'VB'),
 ('any', 'DT'),
 ('moment', 'NN'),
 ('in', 'IN'),
 ('this', 'DT'),
 ('film', 'NN'),
 ('that', 'WDT')]

NLTK identifica cada palabra con una etiqueta. En su ayuda muestra el significado de cada una de las etiquetas.

In [333]:
#nltk.help.upenn_tagset()

En nuestro caso queremos quedarnos con adjetivos, cuyas etiquetas empiezan por __JJ\*__ y los adverbios, que empiezan por __RB\*__ 

In [334]:
[(token, tag) for token, tag in tagged_tokens
 if tag[:2] == 'JJ' or tag[:2] == 'RB'][:10]

[('stupid', 'JJ'),
 ('not', 'RB'),
 ('scary', 'JJ'),
 ('Stupid', 'JJ'),
 ('bet', 'RB'),
 ('Everywhere', 'RB'),
 ('bigger', 'JJR'),
 ('better', 'JJR'),
 ('not', 'RB'),
 ('well', 'RB')]

### Composición de pasos

Tenemos varias ideas para probar con nuestro _vector space model_. Vamos a crear una función que, dependiendo de diferentes parámetros, convierta un texto a la forma deseada. De esta manera vamos a facilitar mucho la experimentación de pruebas a posteriori.

Si en un futuro queremos experimentar con la forma de sacar términos, solo habría que cambiar la siguiente función, dejando el resto del flujo sin modificar.

In [335]:
def custom_tokenizer(text, use_stem=True, stemmer=snow_stemmer, use_pos=False, 
                     use_only_adj=False, use_bigrams=False, use_bigrams_only=False):
    # Separate words
    words = word_tokenize(text)
    # PoS tagging words
    if use_pos:
        pos_tags = nltk.pos_tag(words)
    else:
        pos_tags = zip(words, [''] * len(words))
    
    tokens = []
    # Special treatment for bigrams
    if use_bigrams:
        tokens += list(ngrams(words, n=2))
        if use_bigrams_only:
            return tokens
        else:
            tokens += [(x,) for x in words]
        return tokens
    
    for word, tag in pos_tags:
        res_word = word
        use_word = True
        # Convert to stem
        if use_stem:
            res_word = stemmer.stem(res_word)
        # Use POS tag with the word
        if use_pos and not use_only_adj:
            res_word += '_' + tag
        # Only use adv and adj
        if use_only_adj and not (tag[:2] == 'JJ' or tag[:2] == 'RB'):
            use_word = False
        # Append the word to the tokenizer
        if use_word:
            tokens.append(res_word)
    return tokens

def text_stems_tok(text):
    return custom_tokenizerC
def pos_tok(text):
    return custom_tokenizer(text, use_stem=False, use_pos=True)
def pos_stems_tok(text):
    return custom_tokenizer(text, use_stem=True, use_pos=True)
def adj_tok(text):
    return custom_tokenizer(text, use_stem=False, use_pos=True, use_only_adj=True)
def adj_stems_tok(text):
    return custom_tokenizer(text, use_stem=True, use_pos=True, use_only_adj=True)
def unigrams(text):
    return word_tokenize(text)
def uni_bigrams(text):
    return custom_tokenizer(text, use_bigrams=True)
def bigrams(text):
    return custom_tokenizer(text, use_bigrams=True, use_bigrams_only=True)
def uni_bigrams_stems(text):
    tokens = uni_bigrams(text)
    res_tokens = []
    for t in tokens:
        res_tokens.append(tuple([snow_stemmer.stem(x) for x in t]))
    return res_tokens

In [336]:
features = adj_tok(raw_text)
features[:10]

['stupid',
 'not',
 'scary',
 'Stupid',
 'bet',
 'Everywhere',
 'bigger',
 'better',
 'not',
 'well']

# Creación del _vector space model_

Para la creación y entrenamiento del modelo vamos a usar `scikit-learn`. Scikit-learn ofrece utilidades para la carga de un dataset, creaciones de diferentes tipos de modelos y entrenamientos. Primero cargamos el dataset de entrenamiento y el de test.

In [277]:
from sklearn.datasets import load_files

In [278]:
dataset = load_files('data/train')

In [279]:
test_dataset = load_files('data/test')

Podemos ver que ha generado una clase numérica por cada una de las clases encontradas.

In [71]:
for t in range(len(dataset.target_names)):
    print(t, dataset.target_names[t])

0 neg
1 pos


### Count Vectorizer

La utilidad `CountVectorizer` nos va a permitir transformar nuestros textos a un array de frecuencias de cada uno de los términos de la colección de documentos. Permite incorporar una función personalizada para transformar estos textos en términos y es aquí donde vamos a incorporar nuestras funciones previamente definidas.

Va a convertir cada uno de los textos en una matriz dispersa (_sparse_) donde las columnas representarán los diferentes términos y las columnas los documentos en los que aparecen.

In [280]:
from sklearn.feature_extraction.text import CountVectorizer

In [283]:
count_vect = CountVectorizer(tokenizer=unigrams)
X_train_counts = count_vect.fit_transform(dataset.data)
X_train_counts.shape

(25000, 114666)

### TF-IDF Transformer

Una vez tenemos el conteo de términos por documento es necesario averiguar la frecuencia de estos. Como comentamos antes, no solo vamos a usar la frecuencia (__TF__) sino también la frecuencia inversa (__IDF__). La clase `TfidfTransformer` permite hacer todo esto en un solo paso.

In [284]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(25000, 114666)

### Support Vector Machines como clasificador

Ya tenemos matrices que indican la frecuencia de cada termino menos la frecuencia inversa. Ahora necesitamos entrenar un clasificador que nos permita transformar esta matriz a una de las dos categorías que tenemos: pos o neg.

Aquí podríamos probar varios clasificadores, pero me he centrado en el que mejor me suele funcionar para analisis de frecuencia: __Support Vector Machines SVM__. Una _máquina de soporte vectorial_ es un modelo que representa los puntos de muestra en el espacio, separando las clases a dos espacios lo más amplios posibles mediante un hiperplano.

Es fácil de visualizar cuando los puntos de muestra tienen dos dimensiones, como en la siguiente imagen:

<img src="svm.jpg">

Importamos el clasificador por SVM

In [285]:
from sklearn.linear_model import SGDClassifier

Ahora ajustamos el clasificador a nuestro conjunto de entrenamiento

In [288]:
text_clf = SGDClassifier()
text_clf.fit(X_train_tf, dataset.target)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

Con esto, el modelo está listo para clasificar. Ahora podemos probarlo con algún ejemplo del conjunto de test, pero primero tenemos que transformar el conjunto de test a nuestras features __TF-IDF__.

### SKLearn Pipelines

Cuando tenemos un proceso definido mediantes clases de transformación de sklearn, en vez de tener que recrearlo entero cada vez que queramos trabajar con estos datos podemos definir un _pipeline_. Con este _pipeline_ podremos trabajar como si fuera un modelo único que genera todas las transformaciones y luego clasifica.

Lo interesante de usar este proceso es que al usar input/outputs estandarizados por sklearn, solo haría falta cambiar la línea de 
```
('clf', SGDClassifier())
```
por
```
('clf', MultinomialNB())
```
para usar un clasficador Naive Bayes en vez de una SVM.

In [329]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

text_clf = Pipeline([('vect', CountVectorizer(tokenizer=uni_bigrams_stems)),
                     ('tfidf', TfidfTransformer(use_idf=True)),
                     ('clf', SGDClassifier())
                    ])
                     

Ajustamos el modelo a los datos

In [330]:
text_clf = text_clf.fit(dataset.data, dataset.target)

Comprobamos como se comporta el modelo frente al conjunto de test. Para ello, predecimos todos los documentos del conjunto de test y vemos con cuantos hemos acertado.

In [331]:
import numpy as np
predicted = text_clf.predict(test_dataset.data)
np.mean(predicted == test_dataset.target)

0.89651999999999998

Esto significa que el modelo usando solo bigramas tiene casi un 90% de acierto en el conjunto de test.

# Elección de modelos y cross-validation

Una de las ventajas de usar sklearn es que en vez de probar todas las diferentes funciones que definimos al principio podemos ejecutar una búsqueda para quedarnos con la que tenga mejor puntuación. Para que no triunfe el modelo que mejor se sobreajuste al conjunto de test es interesante hacer cross validación con el conjunto de entrenamiento.

## Cross-validation

La cross validación consiste en dividir el conjunto de entrenamiento en n grupos iguales (en este caso usamos 3 grupos), entrenar el modelo con n-1 grupos y comprobar la eficiencia de este en el grupo restante. Esto se hace n veces y al final podemos saber la puntuación media sobre los n grupos, evitando un posible sobreajuste.

## Búsqueda de parámetros

Para la búsqueda de parámetros y la cross validación usamos la clase GridSearchCV, que nos permite definir un modelo y n parámetros. Entrenará el modelo para todos las combinaciones posibles de esos n parámetros y devolverá el modelo con mejor puntuación.

Teniendo en cuenta que no tenemos un dataset pequeño y que los análisis POS no son rápidos, una búsqueda de parámetros puede ser muy lenta (en mi caso la siguiente búsqueda ha tardado aproximadamente 8 horas).


In [236]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', SGDClassifier()),
                        ])

parameters = {
    'vect__tokenizer': (
        word_tokenize, text_stems_tok, pos_tok,
        pos_stems_tok, adj_tok, adj_stems_tok
    ),
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-3, 1e-4),
}

gs_clf = GridSearchCV(text_clf_svm, parameters, n_jobs=-1)
gs_clf.fit(dataset.data, dataset.target)
predicted = gs_clf.predict(test_dataset.data)
np.mean(predicted == test_dataset.target)

0.88207999999999998

Si importamos los resultados de la búsqueda podemos ver todos los posibles entrenamientos.

In [303]:
import pandas as pd

In [305]:
df = pd.DataFrame(gs_clf.cv_results_)
df.head()

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_clf__alpha,param_tfidf__use_idf,param_vect__tokenizer,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,42.809086,20.595768,0.83835,0.861900,0.001,True,<function word_tokenize at 0x7fd443731e18>,"{'clf__alpha': 0.001, 'vect__tokenizer': <func...",8,0.835158,0.861397,0.835608,0.861697,0.844284,0.862607,0.676449,0.193819,0.004200,0.000515
1,92.014162,44.660706,0.84560,0.864775,0.001,True,<function text_stems_tok at 0x7fd434280bf8>,"{'clf__alpha': 0.001, 'vect__tokenizer': <func...",5,0.838908,0.867622,0.846108,0.867247,0.851785,0.859457,6.561049,0.289353,0.005269,0.003764
2,291.978515,156.644288,0.82990,0.853275,0.001,True,<function pos_tok at 0x7fd434280c80>,"{'clf__alpha': 0.001, 'vect__tokenizer': <func...",11,0.832758,0.859446,0.826459,0.854571,0.830483,0.845808,8.023300,4.274308,0.002605,0.005643
3,361.926162,183.052437,0.82890,0.853500,0.001,True,<function pos_stems_tok at 0x7fd434280ea0>,"{'clf__alpha': 0.001, 'vect__tokenizer': <func...",12,0.826909,0.858171,0.828409,0.855921,0.831383,0.846408,5.289329,11.005596,0.001859,0.005099
4,299.924710,149.129380,0.82605,0.848325,0.001,True,<function adj_tok at 0x7fd434280d90>,"{'clf__alpha': 0.001, 'vect__tokenizer': <func...",14,0.825109,0.848796,0.823309,0.849696,0.829733,0.846483,7.587796,4.655080,0.002706,0.001354


Los resultados para algunos de los modelos son los siguientes

|                      | TF-IDF  | TF      |
|----------------------|---------|---------|
| Unigram              | 0.88670 | 0.80215 |
| Unigram with stems   | 0.88765 | 0.81555 |
| Bigram               | 0.88351 | __0.85972__ |
| Unigram + Bigram     | 0.89459 | --      |
| Unigram + Bigram with stems    | __0.89651__ | --      |
| POS                  | 0.88350 | 0.81745 |
| POS with stems       | 0.88456 | 0.80490 |
| Adj + adv            | 0.84435 | 0.83235 |
| Adj + adv with stems | 0.84070 | 0.83200 |

Como podemos ver, el mejor modelo lo ha conseguido es el que combina unigramas con bigramas de las raices de las palabras. En uno de los artículos donde usan este dataset \[2\] la mejor puntuación es de __0.89632__ usando unigramas + bigramas, mientras que la mía es __0.89651__ con la misma técnica pero usando las raices de los tokens.

Como dato interesante, si no usamos __IDF__ nos estamos quedando sin mecanismo para eliminar palabras comunes como _the_ o signos de puntuación. No es raro entonces que en este caso los mejores modelos sean aquellos que ya incorporan algún tipo de filtro, como los bigramas (es raro que haya dos palabras de este tipo seguidas) y cuando usamos solo adjetivos y advervios.

# Guardar y cargar el modelo

Como hemos visto a veces los modelos son muy lentos de entrenar, por lo que es conveniente guardarlos a disco una vez están entrenados. De esta manera, cuando haya que usarlos solo hay que cargarlos sin tener que re-entrenar nada

In [318]:
from sklearn.externals import joblib

In [325]:
# Save the model
joblib.dump(text_clf, 'my_model.pkl')

['my_model.pkl']

In [326]:
# Load the model
loaded_clf = joblib.load('my_model.pkl')

# Bibliografía

* \[1\] [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/)
* \[2\] [Sentiment analysis of users' reviews and comments](http://cs229.stanford.edu/proj2012/ChakankarVenuturimilliMathur-SentimentAnalysis.pdf)
* \[3\] [NLTK Book](http://www.nltk.org/book/)
* \[4\] Statistics: The vector space model (J. F. Quesada)
* \[5\] [Scikit-learn: Working with text data](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)
* \[6\] [Wikipedia: Sentiment analysis](https://en.wikipedia.org/wiki/Sentiment_analysis)
